In [18]:
# HElP about the coordinates can be found in https://ratschlab.slack.com/archives/DD4F40KPF/p1648498023344119 (DM LP/ AK)
# /cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.hdf5 and
# /cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle
# There, I looked for gene ENSG00000085733.16, which has ID 9803. Then I checked which edge in the segmentgraph corresponds to 70435040;70436454 , which are segments 169 (array([70435025, 70435040])) and 186 (array([70436454, 70436562])). This translates to a multi_index integer of 32296. Checking in the correct range of the hdf5 where the gene_ids_edges is 9803 for the edge_idx of 32296, gets an offset value of 175. Long story short, the edge corresponds to the count matrix entry edges[221005, :] . In this count vector only a single index has a non-zero value (index 855), which corresponds to sample TCGA-E2-A1B4-01A-11R-A12P-07.all. In this sample I could also find on the single sample level a junction count of 2 for this junctions. For all other samples (I used a bash loop), I was not able to find support for the junction.

In [19]:
# Pseudo code to get a junction expression using the graph and the count file

# 1. GET THE EDGES_INDEXES that correspond to the gene of interest VIA: 
# edges_index_gene = [i for i, j in enumerate(h5f['gene_ids_edges'][:, 0]) if j == gene_hd5_id]

# 2. GET the multi_indexes
# SLICE h5f['edge_idx'] for the EDGES_INDEXES above: 
# multi_indexes = h5f['edge_idx'][edges_index_gene] 

# 3. THE multi_indexes obained match perfectly the ones in np.where(gene.segmentgraph.seg_edges.flatten())[0]

# REMARK: HISTORICALLY I used to work more whith the offsets that the multi_indexes
# Can be seen this way
#print([ i for i, foo in enumerate(zip(np.where(gene.segmentgraph.seg_edges)[0], np.where(gene.segmentgraph.seg_edges)[1])) 
#if (foo[0] == 169) & (foo[1] == 186)]) # THIS IS THE OFFSET
# The offset is directly associated with the non-flattened structure. 

# 4. TO get an offset: 
#offset = [myid for myid, edge_idx in enumerate(h5f['edge_idx'][edges_index_gene]) if edge_idx == multi_index_to_locate][0]  
# OR 
# offset = [myid for myid, edge_idx in enumerate(multi_indexes) if edge_idx == multi_index_to_locate][0]  
# 5. We can check that h5f['edge_idx'][edges_index_gene][offset] = multi_index_to_locate

# TO GET the EDGE index of my favorite junction (absolute_pos) from the multi_index_to_locate
#absolute_pos = [myid for myid, edge_idx in enumerate(h5f['edge_idx']) if edge_idx == multi_index_to_locate][0]


# TO GET the EDGE index of my favorite junction (absolute_pos) from the offset
# edges_index_gene[offset]

# REMARK: We can check that h5f['edge_idx'][absolute_pos] = multi_index_to_locate

# TO GET the EDGE EXPRESSION of of my favorite junction: 
# h5f['edges'][absolute_pos, :] # all samples

# TO GET the EDGE EXPRESSION of all junctions for a given sample
#h5f['edges'][edges_index_gene, sample]


In [20]:
import os 
import pandas as pd
import path
import pickle 
import numpy as np
import h5py

In [21]:
### INPUTS PARAMETERS
sample_type = 'OV' # BRCA
file_type = 'unfiltered' #'conf2'
sample = 'sample1' # relink samples 1 to 5

In [22]:
### INPUTS FILES
base_path_junctions = '/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes'
## Spladder junction files
# OV
folder_OV_jx_unfiltered = 'TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv'
folder_OV_jx_conf2 = 'TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv'

# BRCA
folder_BRCA_jx_unfiltered = 'TCGA_Breast_1102_results/TCGA_Breast_1102_results_junctions_unfiltered_tsv'
folder_BRCA_jx_conf2 = 'TCGA_Breast_1102_results/TCGA_Breast_1102_results_junctions_conf_2_tsv'

## Spladder Graphs
# OV
count_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.rechunked.hdf5')
#count_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.hdf5')
splice_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle')
sample_file_OV = '/cluster/work/grlab/projects/projects2020_OHSU/sample_lists/TCGA_foreground/sample_full_Ov_378.tsv'
output_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/junctions_from_graph.tsv.gz')

# BRCA
count_path_BRCA = os.path.join( base_path_junctions, 'TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.hdf5')
splice_path_BRCA = os.path.join( base_path_junctions, 'TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle')
sample_file_BRCA = "/cluster/work/grlab/projects/projects2020_OHSU/sample_lists/TCGA_foreground/sample_full_BRCA_1102.tsv"
output_path_BRCA = os.path.join( base_path_junctions, 'TCGA_Breast_1102_results/splicing/junctions_from_graph.tsv.gz')

In [23]:
# Paths options
if sample_type == 'OV':
    splice_path = splice_path_OV
    count_path = count_path_OV

    folder_jx_unfiltered = folder_OV_jx_unfiltered
    folder_jx_conf2 = folder_OV_jx_conf2
    output_path_ = output_path_OV

elif sample_type == 'BRCA':
    splice_path = splice_path_BRCA
    count_path = count_path_BRCA

    folder_jx_unfiltered = folder_BRCA_jx_unfiltered
    folder_jx_conf2 = folder_BRCA_jx_conf2 # TODO + GET TABLES FROM MARIIA's files
    output_path_ = output_path_BRCA
    
path_jx_unfiltered = os.path.join(base_path_junctions, folder_jx_unfiltered, sample)
print(path_jx_unfiltered)
path_jx_conf2 = os.path.join(base_path_junctions, folder_jx_conf2, sample)
print(path_jx_conf2)
path_jx_unfiltered = os.path.realpath(path_jx_unfiltered)
print(path_jx_unfiltered)
path_jx_conf2 = os.path.realpath(path_jx_conf2)
print(path_jx_conf2)

/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv/sample1
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv/sample1
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv/TCGA-25-1319-01A-01R-1565-13.all.junctions.tsv.gz
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv/TCGA-25-1319-01A-01R-1565-13.all.conf_2.filt.junctions.tsv.gz


In [24]:
# Get sample name
sample_name = path_jx_unfiltered.split('/')[-1].replace('.junctions.tsv.gz', '')
print(sample_name)

TCGA-25-1319-01A-01R-1565-13.all


In [25]:
output_path = output_path_.replace('.tsv.gz', f'.{sample_name}.tsv.gz')
print(output_path)

/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/splicing/junctions_from_graph.TCGA-25-1319-01A-01R-1565-13.all.tsv.gz


# "Spladder files"

In [26]:
# Load spladder files
df_jx_unfiltered = pd.read_csv(path_jx_unfiltered,
                                  sep = '\t')
df_jx_conf2 = pd.read_csv(path_jx_conf2, 
                                  sep = '\t')

if file_type == 'unfiltered':
    df_spladder_junctions = df_jx_unfiltered
elif file_type == 'conf2':
    df_spladder_junctions = df_jx_conf2

In [27]:
def unique_junctions_df(df):
    display(df.head(3))
    print('shape table')
    print(df.shape)
    print('shape table excluding column count')
    print(df.drop(['count'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, chr')
    print(df.drop(['count', 'chr'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, strand')
    print(df.drop(['count', 'strand'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, strand, chr')
    print(df.drop(['count', 'strand', 'chr'], axis = 1).drop_duplicates().shape)
    

In [28]:
# Initial analysis of unique junctions
print(path_jx_unfiltered)
unique_junctions_df(df_jx_unfiltered)
print(path_jx_conf2)
unique_junctions_df(df_jx_conf2)

/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv/TCGA-25-1319-01A-01R-1565-13.all.junctions.tsv.gz


,chr,strand,junction_start,junction_end,count
0,chr10,-,48114,48614,1
1,chr10,-,48725,48803,3
2,chr10,-,277577,281199,12


shape table
(485934, 5)
shape table excluding column count
(485934, 4)
shape table excluding column count, chr
(485747, 3)
shape table excluding column count, strand
(485436, 3)
shape table excluding column count, strand, chr
(485249, 2)
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv/TCGA-25-1319-01A-01R-1565-13.all.conf_2.filt.junctions.tsv.gz


,chr,strand,junction_start,junction_end,count
0,chr10,-,48114,48614,1
1,chr10,-,48725,48803,1
2,chr10,-,277577,281199,9


shape table
(396650, 5)
shape table excluding column count
(396650, 4)
shape table excluding column count, chr
(396510, 3)
shape table excluding column count, strand
(396405, 3)
shape table excluding column count, strand, chr
(396265, 2)


# Graph ALL Genes

In [ ]:
# Load graph
graph, meta = pickle.load(open(splice_path, 'rb'))

In [ ]:
# Load count file 
h5f = h5py.File(count_path, 'r')
print(h5f.keys())
for k in h5f.keys():
    print(k)
    print(h5f[k].shape)
    # Subset sample
    sample_id = [i for i, j in enumerate(h5f['strains']) if j.decode() == sample_name][0]
    edge_expr_sample = h5f['edges'][:, sample_id]


In [ ]:
prepare_df = []

for idx, gene in enumerate(graph):
    if idx % 1000 == 0:
        print(idx)
    gene.from_sparse()

    # Graph segment graph indexes and coord
    seg_graph_index = np.array([(a,b) for a, b in zip(np.where(gene.segmentgraph.seg_edges)[0],
                                                   np.where(gene.segmentgraph.seg_edges)[1])])
    segms_gene = [(gene.segmentgraph.segments[1][i], gene.segmentgraph.segments[0][j]) 
                  for i, j in seg_graph_index]

    # Count file gene ID
    gene_hd5_id = idx #[i for i, j in enumerate(h5f['gene_names'][:, 0]) if j.decode() == gene.name][0] 
    # Count file edges index
    edges_index_gene = np.where(h5f['gene_ids_edges'][:, 0] == gene_hd5_id)[0]
    # Count file get expression vector 
    edge_expr_sample_gene = edge_expr_sample[edges_index_gene]


    # Create data frame with the junction positions and the expression
    all_junctions_coord_with_expr = [[jx[0], jx[1], expr] 
                          for jx, expr in zip(segms_gene, edge_expr_sample_gene)]
    all_expr_df_graph = pd.DataFrame(all_junctions_coord_with_expr, 
                                     columns = ['junction_start', 'junction_end', 'count_graph'])
    all_expr_df_graph = all_expr_df_graph.drop_duplicates()
    all_expr_df_graph['gene_graph'] = gene.name
    all_expr_df_graph['strand_graph'] = gene.strand
    
    prepare_df.append(all_expr_df_graph)

In [ ]:
# Merge
all_expr_df_graph = pd.concat(prepare_df)

print(all_expr_df_graph.shape)

display(all_expr_df_graph.head())

In [ ]:
# Save 
all_expr_df_graph.to_csv(output_path, sep = '\t', index=None, compression='gzip')
print(output_path)

In [ ]:
# CLOSE count file
h5f.close()

# Merge

In [32]:
# TODO - DO TWO MERGES FOR EACH OF THE SPLADDER FILES FILES..

In [33]:
#Load 
all_expr_df_graph = pd.read_csv(output_path, sep = '\t')
print(all_expr_df_graph.shape)

display(all_expr_df_graph.head())

(1979609, 5)


,junction_start,junction_end,count_graph,gene_graph,strand_graph
0,11211,12009,0.0,ENSG00000223972.5,+
1,11671,12009,0.0,ENSG00000223972.5,+
2,11844,12009,0.0,ENSG00000223972.5,+
3,12057,12178,0.0,ENSG00000223972.5,+
4,12227,12496,0.0,ENSG00000223972.5,+


In [34]:
# Merge Spladder junctions to Graph Junctions
df_spladder_junctions = df_spladder_junctions.rename({'count': 'count_tsv_spladder'}, axis = 1)
df_spladder_junctions = df_spladder_junctions.rename({'strand': 'strand_tsv_spladder'}, axis = 1)
df_spladder_junctions = df_spladder_junctions.rename({'chr': 'chr_tsv_spladder'}, axis = 1)
merged_df = all_expr_df_graph.merge(df_spladder_junctions, 
                                    on = ['junction_start', 'junction_end'], how = 'outer')

In [35]:
merged_df.head()

,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder
0,11211,12009,0.0,ENSG00000223972.5,+,NaN,NaN,NaN
1,11671,12009,0.0,ENSG00000223972.5,+,NaN,NaN,NaN
2,11844,12009,0.0,ENSG00000223972.5,+,NaN,NaN,NaN
3,12057,12178,0.0,ENSG00000223972.5,+,NaN,NaN,NaN
4,12227,12496,0.0,ENSG00000223972.5,+,NaN,NaN,NaN


In [36]:
all_expr_df_graph.head()

,junction_start,junction_end,count_graph,gene_graph,strand_graph
0,11211,12009,0.0,ENSG00000223972.5,+
1,11671,12009,0.0,ENSG00000223972.5,+
2,11844,12009,0.0,ENSG00000223972.5,+
3,12057,12178,0.0,ENSG00000223972.5,+
4,12227,12496,0.0,ENSG00000223972.5,+


In [37]:
print('spladder junctions tsv', df_spladder_junctions.shape)
print('graph', all_expr_df_graph.shape)
display(merged_df.head())

print('merged', merged_df.shape)
print(merged_df.drop_duplicates().shape)


spladder junctions tsv (485934, 5)
graph (1979609, 5)


,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder
0,11211,12009,0.0,ENSG00000223972.5,+,NaN,NaN,NaN
1,11671,12009,0.0,ENSG00000223972.5,+,NaN,NaN,NaN
2,11844,12009,0.0,ENSG00000223972.5,+,NaN,NaN,NaN
3,12057,12178,0.0,ENSG00000223972.5,+,NaN,NaN,NaN
4,12227,12496,0.0,ENSG00000223972.5,+,NaN,NaN,NaN


merged (2158986, 8)
(2158986, 8)


# Visualisation 

In [79]:
print(merged_df.loc[(~ merged_df['count_graph'].isna()) | (~ merged_df['count_tsv_spladder'].isna()), 
                ['junction_start', 'junction_end']].drop_duplicates().shape)

(2149521, 2)


In [80]:
# One is assigned 0 expression and the other one is assigned a higher expression
merged_df_not_NAs = merged_df[(~ merged_df['count_graph'].isna()) & (~ merged_df['count_tsv_spladder'].isna())]
divergence_1 = merged_df_not_NAs[     (merged_df_not_NAs['count_graph'] == 0 ) 
                                    &  (merged_df_not_NAs['count_tsv_spladder']!= 0 ) ]
display(divergence_1.head(3))
print(divergence_1.shape)
divergence_1_shape = divergence_1[['junction_start', 'junction_end']].drop_duplicates().shape[0]


divergence_2 = merged_df_not_NAs[     (merged_df_not_NAs['count_tsv_spladder'] == 0 ) 
                                    &  (merged_df_not_NAs['count_graph']!= 0 ) ]
display(divergence_2.head(3))
print(divergence_2.shape)
divergence_2_shape = divergence_2[['junction_start', 'junction_end']].drop_duplicates().shape[0]

,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder
4592,1312846,1313039,0.0,ENSG00000127054.20,-,chr1,+,2.0
11385,2590733,2590933,0.0,ENSG00000157870.16,+,chr1,-,1.0
17887,8862939,8865294,0.0,ENSG00000074800.16,-,chr1,+,1.0


(434, 8)


,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder


(0, 8)


In [81]:
# BOTH ARE NAs
merged_df.loc[ ( merged_df['count_graph'].isna())  &
               ( merged_df['count_tsv_spladder'].isna() )]

,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder


In [82]:
# Expressed in any of the two files
union = merged_df.loc[ ( (~ merged_df['count_graph'].isna()) & merged_df['count_graph'] != 0) |
              (  (~ merged_df['count_tsv_spladder'].isna()) &  merged_df['count_tsv_spladder']!= 0 )]
print(union.shape)
display(union.head(3))
union_shape = union[['junction_start', 'junction_end']].drop_duplicates().shape[0]

(492884, 8)


,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder
10,12721,13220,1.0,ENSG00000223972.5,+,chr1,+,1.0
31,14829,14969,104.0,ENSG00000227232.5,-,chr1,-,104.0
40,15038,15795,15.0,ENSG00000227232.5,-,chr1,-,15.0


In [83]:
# Expressed in both files
inter = merged_df.loc[ ( (~ merged_df['count_graph'].isna()) & merged_df['count_graph'] != 0) &
              (  (~ merged_df['count_tsv_spladder'].isna()) &  merged_df['count_tsv_spladder']!= 0 )]
print(inter.shape)
display(inter.head(3))
inter_shape = inter[['junction_start', 'junction_end']].drop_duplicates().shape[0]

(313523, 8)


,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder
10,12721,13220,1.0,ENSG00000223972.5,+,chr1,+,1.0
31,14829,14969,104.0,ENSG00000227232.5,-,chr1,-,104.0
40,15038,15795,15.0,ENSG00000227232.5,-,chr1,-,15.0


In [84]:
# Expressed and not present in the other file
spladder_counts_only = merged_df.loc[merged_df['count_graph'].isna() & (merged_df['count_tsv_spladder'] != 0 )]
print(spladder_counts_only.shape)
display(spladder_counts_only.head(3))
spladder_counts_only_shape = spladder_counts_only[['junction_start', 'junction_end']].drop_duplicates().shape[0]

(178927, 8)


,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder
1980059,804360,813084,NaN,NaN,NaN,chr10,-,12.0
1980060,817841,825178,NaN,NaN,NaN,chr10,-,1.0
1980061,885653,1071309,NaN,NaN,NaN,chr10,-,1.0


In [85]:
# Expressed and not present in the other file
graph_only =  merged_df.loc[merged_df['count_tsv_spladder'].isna() & (merged_df['count_graph'] != 0 )]
print(graph_only.shape)
display(graph_only.head())
graph_only_shape = graph_only[['junction_start', 'junction_end']].drop_duplicates().shape[0]

(0, 8)


,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder


In [86]:
inter.shape[0] + graph_only.shape[0] + spladder_counts_only.shape[0] \
+ divergence_1.shape[0] + divergence_2.shape[0] == union.shape[0]

True

In [91]:
inter_shape + graph_only_shape + spladder_counts_only_shape \
+ divergence_1_shape + divergence_2_shape == union_shape

False

In [94]:
print(inter.shape[0])
print(graph_only.shape[0] )
print(spladder_counts_only.shape[0])
print( divergence_1.shape[0])
print(divergence_2.shape[0])
print(union.shape[0])

313523
0
178927
434
0
492884


In [93]:
print(inter_shape)
print(graph_only_shape)
print(spladder_counts_only_shape)
print(divergence_1_shape)
print(divergence_2_shape)
print(union_shape)

306390
0
178537
434
0
485249


In [101]:
485249 - (306390 + 178537)

322

In [92]:
print(inter.shape[0] + graph_only.shape[0])
print(inter.shape[0] + graph_only.shape[0] +  divergence_1.shape[0])
print(inter.shape[0] + spladder_counts_only.shape[0])
print(inter.shape[0] + spladder_counts_only.shape[0] +  divergence_1.shape[0])


313523
313957
492450
492884


In [97]:
unique_junctions_df(df_jx_unfiltered)

,chr,strand,junction_start,junction_end,count
0,chr10,-,48114,48614,1
1,chr10,-,48725,48803,3
2,chr10,-,277577,281199,12


shape table
(485934, 5)
shape table excluding column count
(485934, 4)
shape table excluding column count, chr
(485747, 3)
shape table excluding column count, strand
(485436, 3)
shape table excluding column count, strand, chr
(485249, 2)


In [102]:
# START RES

In [72]:
res.loc[res['_merge'] == 'right_only']

,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder,chr,strand,count,_merge
494130,80169100,80169158,NaN,NaN,NaN,NaN,NaN,NaN,chr10,+,1,right_only
494131,308187,308316,NaN,NaN,NaN,NaN,NaN,NaN,chr11,-,1,right_only
494132,46382444,46382662,NaN,NaN,NaN,NaN,NaN,NaN,chr11,-,1,right_only
494133,57699964,57700044,NaN,NaN,NaN,NaN,NaN,NaN,chr11,-,1,right_only
494134,67586163,67586444,NaN,NaN,NaN,NaN,NaN,NaN,chr11,-,1,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...
494447,1387290,1389336,NaN,NaN,NaN,NaN,NaN,NaN,chrX,-,1,right_only
494448,1389317,1389647,NaN,NaN,NaN,NaN,NaN,NaN,chrX,-,1,right_only
494449,48823270,48823344,NaN,NaN,NaN,NaN,NaN,NaN,chrX,-,1,right_only
494450,135032734,135032812,NaN,NaN,NaN,NaN,NaN,NaN,chrX,-,1,right_only


In [ ]:
# END TEST

# Special Junctions Cases

In [47]:
j1=30338876
j12=30346441
merged_df.loc[(merged_df['junction_start'] == j1) & (merged_df['junction_end'] == j12)]

,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder
2093635,30338876,30346441,NaN,NaN,NaN,chr22,-,1.0


In [48]:
ja=39630003
jb=39634028
merged_df.loc[(merged_df['junction_start'] == ja) & (merged_df['junction_end'] == jb)]

,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder
2029191,39630003,39634028,NaN,NaN,NaN,chr17,+,1.0


In [49]:
j1=53228131
j12=53240255
merged_df.loc[(merged_df['junction_start'] == j1) & (merged_df['junction_end'] == j12)]

,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder
1377889,53228131,53240255,1.0,ENSG00000163931.16,-,chr3,-,1.0


In [50]:
ja=44813454
jb=44819345
merged_df.loc[(merged_df['junction_start'] == ja) & (merged_df['junction_end'] == jb)]



,junction_start,junction_end,count_graph,gene_graph,strand_graph,chr_tsv_spladder,strand_tsv_spladder,count_tsv_spladder
1012076,44813454,44819345,1.0,ENSG00000187244.11,+,chr19,+,1.0
